In [ ]:
import os
import numpy as np
import cv2 as cv
import gc
from tqdm.notebook import tqdm

In [ ]:
# Keras/TensorFlow Imports
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomZoom, RandomTranslation
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [ ]:
# -------------------------------------------------------------------
# 1. SETUP VARIABLES AND DATA PATHS
# -------------------------------------------------------------------
IMG_SIZE = (80, 80)
channels = 1  # 1 for grayscale, 3 for color (RGB)
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

In [ ]:
# Creating a character dictionary and sorting it
char_dict = {}
for char in os.listdir(char_path):
    # Only process directories
    if os.path.isdir(os.path.join(char_path, char)):
        try:
            char_dict[char] = len(os.listdir(os.path.join(char_path, char)))
        except Exception:
            # Handle permissions or other errors if necessary
            pass

# Sort in descending order (Standard Python Replacement for caer.sort_dict)
sorted_char_dict_list = sorted(char_dict.items(), key=lambda item: item[1], reverse=True)

In [ ]:
# Select the top 10 characters to use as classes (common for this dataset)
characters = [item[0] for item in sorted_char_dict_list[:10]]
num_classes = len(characters)

In [ ]:
print(f"Number of classes selected: {num_classes}")
print(f"Classes: {characters}")
print("-" * 30)

In [ ]:
# -------------------------------------------------------------------
# 2. CUSTOM PREPROCESSING FUNCTION (Replacement for caer.preprocess_from_dir)
# -------------------------------------------------------------------

def preprocess_images_custom(DIR, classes, IMG_SIZE, channels):
    data = []
    
    # Create a mapping from class name to numerical index (0 to num_classes-1)
    class_to_index = {class_name: i for i, class_name in enumerate(classes)}
    
    print("[INFO] Starting preprocessing...")

    for class_name in tqdm(classes, desc="Processing Classes"):
        class_path = os.path.join(DIR, class_name)
        class_index = class_to_index[class_name]
        
        if not os.path.exists(class_path):
            continue
            
        # Iterate over all files in the class directory
        for img_name in os.listdir(class_path):
            try:
                img_path = os.path.join(class_path, img_name)
                
                # Load image using OpenCV
                # Use cv.IMREAD_GRAYSCALE for channels=1, or cv.IMREAD_COLOR for channels=3
                if channels == 1:
                    img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
                else:
                    # By default, load color and convert to RGB (OpenCV loads BGR)
                    img = cv.imread(img_path)
                    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
                
                if img is None:
                    continue
                    
                # Resize image
                resized_img = cv.resize(img, IMG_SIZE)
                
                # Append the image (features) and the class index (label)
                data.append([resized_img, class_index])
                
            except Exception as e:
                # print(f"Error loading {img_name}: {e}")
                pass
    
    print("[INFO] Preprocessing complete. Shuffling data...")
    # Shuffle the data
    np.random.shuffle(data)
    
    # Separate features (X) and labels (y)
    X = np.array([item[0] for item in data])
    y = np.array([item[1] for item in data])
    
    # Reshape features to (N, H, W, C)
    # The -1 means infer the number of samples (N)
    X = X.reshape(-1, IMG_SIZE[0], IMG_SIZE[1], channels)
    
    # Normalize features
    X = X.astype('float32') / 255.0
    
    # Convert labels to categorical (one-hot encoding)
    y = to_categorical(y, num_classes=len(classes))
    
    print(f"[INFO] Features shape: {X.shape}")
    print(f"[INFO] Labels shape: {y.shape}")
    
    return X, y

In [ ]:
# -------------------------------------------------------------------
# 3. IMPROVED MODEL FUNCTION - REDUCED COMPLEXITY + DATA AUGMENTATION
# -------------------------------------------------------------------

def create_improved_simpsons_classifier(IMG_SIZE, channels, num_classes):
    """
    Improved model with:
    - Data augmentation layers
    - Reduced model complexity (fewer filters)
    - Increased dropout rates
    - Batch normalization for stability
    """
    model = Sequential()
    
    # DATA AUGMENTATION LAYERS (applied during training only)
    model.add(RandomFlip("horizontal", input_shape=(IMG_SIZE[0], IMG_SIZE[1], channels)))
    model.add(RandomRotation(0.15))  # ±15 degrees
    model.add(RandomZoom(0.1))  # ±10% zoom
    model.add(RandomTranslation(height_factor=0.1, width_factor=0.1))  # ±10% translation
    
    # CONVOLUTIONAL LAYERS - REDUCED COMPLEXITY
    # Layer 1: 16 filters (reduced from 32)
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(16, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))  # Increased from 0.25
    
    # Layer 2: 32 filters (reduced from 64)
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))  # Increased from 0.25

    # Layer 3: 64 filters (reduced from 128)
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(BatchNormalization())
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.4))  # Increased from 0.25
    
    # CLASSIFICATION HEAD - REDUCED COMPLEXITY
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))  # Reduced from 512
    model.add(BatchNormalization())
    model.add(Dropout(0.6))  # Increased from 0.5
    model.add(Dense(num_classes, activation='softmax'))  # Output layer
    
    # Compile the model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

In [ ]:
# -------------------------------------------------------------------
# 4. EXECUTION
# -------------------------------------------------------------------

# 4.1 Create the training data
X_train, y_train = preprocess_images_custom(char_path, characters, IMG_SIZE, channels)

# Clear memory
gc.collect()

In [ ]:
# 4.2 Create the improved model
model = create_improved_simpsons_classifier(IMG_SIZE, channels, num_classes)

# Display model architecture
print("="*50)
print("IMPROVED MODEL ARCHITECTURE")
print("="*50)
model.summary()
print("="*50)

In [ ]:
# 4.3 Setup training configuration with callbacks
BATCH_SIZE = 32
EPOCHS = 30  # Increased from 10, but early stopping will prevent overfitting

# CALLBACKS FOR BETTER TRAINING
callbacks = [
    # Early Stopping: Stop training when validation loss stops improving
    EarlyStopping(
        monitor='val_loss',
        patience=5,  # Stop if no improvement for 5 epochs
        restore_best_weights=True,
        verbose=1
    ),
    
    # Model Checkpoint: Save the best model based on validation accuracy
    ModelCheckpoint(
        'simpsons_classifier_best.h5',
        monitor='val_accuracy',
        save_best_only=True,
        mode='max',
        verbose=1
    ),
    
    # Reduce Learning Rate: Reduce LR when validation loss plateaus
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,  # Reduce LR by half
        patience=3,  # Wait 3 epochs before reducing
        min_lr=1e-7,
        verbose=1
    )
]

print("="*50)
print("TRAINING CONFIGURATION")
print("="*50)
print(f"Batch Size: {BATCH_SIZE}")
print(f"Max Epochs: {EPOCHS}")
print(f"Validation Split: 20%")
print(f"Callbacks: Early Stopping, Model Checkpoint, ReduceLROnPlateau")
print("="*50)

In [ ]:
# 4.4 Train the model
print("\n" + "="*50)
print("STARTING TRAINING")
print("="*50)

history = model.fit(
    X_train, y_train,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_split=0.2,
    callbacks=callbacks,
    verbose=1
)

print("\n" + "="*50)
print("TRAINING COMPLETE")
print("="*50)

In [ ]:
# 4.5 Plot training history
import matplotlib.pyplot as plt

# Plot accuracy
plt.figure(figsize=(14, 5))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

# Plot loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss Over Epochs')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# Print final metrics
print("\n" + "="*50)
print("FINAL TRAINING METRICS")
print("="*50)
print(f"Final Training Accuracy: {history.history['accuracy'][-1]:.4f} ({history.history['accuracy'][-1]*100:.2f}%)")
print(f"Final Validation Accuracy: {history.history['val_accuracy'][-1]:.4f} ({history.history['val_accuracy'][-1]*100:.2f}%)")
print(f"Final Training Loss: {history.history['loss'][-1]:.4f}")
print(f"Final Validation Loss: {history.history['val_loss'][-1]:.4f}")
print(f"Accuracy Gap: {(history.history['accuracy'][-1] - history.history['val_accuracy'][-1])*100:.2f}%")
print("="*50)

In [ ]:
# Clear training data from memory
del X_train
del y_train
gc.collect()

In [ ]:
# 4.6 Prediction Utility (Replacement for 'prepare' function using caer)
def prepare_image(img_path, IMG_SIZE, channels):
    try:
        if channels == 1:
            img = cv.imread(img_path, cv.IMREAD_GRAYSCALE)
        else:
            img = cv.imread(img_path)
            img = cv.cvtColor(img, cv.COLOR_BGR2RGB)  # Convert BGR to RGB
            
        if img is None:
            raise FileNotFoundError(f"Image not found at {img_path}")
            
        resized_img = cv.resize(img, IMG_SIZE)
        
        # Reshape for Keras (1, H, W, C) and normalize
        reshaped_img = resized_img.reshape(1, IMG_SIZE[0], IMG_SIZE[1], channels) / 255.0
        
        return reshaped_img
    except Exception as e:
        print(f"Error in prepare_image: {e}")
        return None

In [ ]:
# 4.7 Example Prediction and Saving
# Example test image path - uses first image from first character class
test_img_path = os.path.join(char_path, characters[0], os.listdir(os.path.join(char_path, characters[0]))[0])

# Prepare the image
prepared_img = prepare_image(test_img_path, IMG_SIZE, channels)

if prepared_img is not None:
    # Make prediction
    predictions = model.predict(prepared_img)

    # Get class with the highest probability
    predicted_index = np.argmax(predictions[0])
    predicted_class = characters[predicted_index]
    confidence = predictions[0][predicted_index]
    
    print("\n" + "="*50)
    print("SAMPLE PREDICTION")
    print("="*50)
    print(f"Test Image: {test_img_path}")
    print(f"Predicted Character: {predicted_class}")
    print(f"Confidence: {confidence*100:.2f}%")
    print("="*50)

    # Save the final model
    model.save('simpsons_classifier.h5')
    print("\n[INFO] Model saved to 'simpsons_classifier.h5'")
    print("[INFO] Best model saved to 'simpsons_classifier_best.h5' (via ModelCheckpoint)")
else:
    print("Skipping prediction and saving due to image loading error.")